In [2]:
import pandas as pd
import numpy as np

In [3]:
#import dataset
data = pd.read_csv("C:/Users/user/NOTEBOOKS/EDA & Vizualization/netflix_titles.csv")
# printing 5 rows
data.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8807 non-null   object
 1   type          8807 non-null   object
 2   title         8807 non-null   object
 3   director      6173 non-null   object
 4   cast          7982 non-null   object
 5   country       7976 non-null   object
 6   date_added    8797 non-null   object
 7   release_year  8807 non-null   int64 
 8   rating        8803 non-null   object
 9   duration      8804 non-null   object
 10  listed_in     8807 non-null   object
 11  description   8807 non-null   object
dtypes: int64(1), object(11)
memory usage: 825.8+ KB


In [5]:
#fetch columns of the dataset
columns = list(data.columns)
columns

['show_id',
 'type',
 'title',
 'director',
 'cast',
 'country',
 'date_added',
 'release_year',
 'rating',
 'duration',
 'listed_in',
 'description']

In [6]:
#examine missing values
print("Missing values distribution: ")

#Mask all values that are NaN as True
#compute the mean of Boolean mask (True evaluates as 1 and False as 0)
res = data.isnull().mean().sort_values(ascending = False)
print(res)

Missing values distribution: 
director        0.299080
country         0.094357
cast            0.093675
date_added      0.001135
rating          0.000454
duration        0.000341
show_id         0.000000
type            0.000000
title           0.000000
release_year    0.000000
listed_in       0.000000
description     0.000000
dtype: float64


#### director column has the highest percentage of missing data ~ 30%

#### cast and country column also has a considerable percentage of missing data ~ 9%

#### date_added, rating and duration don’t have that much missing data ~ 0% - 0.1%

#### Other columns are not empty.

In [7]:
# check datatype in each column
print("Column datatypes: ")
print(data.dtypes)

Column datatypes: 
show_id         object
type            object
title           object
director        object
cast            object
country         object
date_added      object
release_year     int64
rating          object
duration        object
listed_in       object
description     object
dtype: object


In [8]:
# getting all the columns with string/mixed type values
str_cols = list(data.columns)
str_cols.remove('release_year')

# removing leading and trailing characters from columns with str type
for i in str_cols:
    #print(type(data[i]))
    data[i] = data[i].str.strip()

#### Handling missing values(NaN)

#### We have following object types:

String: director, cast, country, rating

Mixed: date_added, duration

In [9]:
columns = ['director','cast','country','rating','date_added']

# looping through the columns to fill the entries with NaN values with ""
for column in columns:
    data[column].fillna("", inplace = True) 

##### Extracting year and month from date for further trend analysis

In [10]:
# examining rows with null values for date_added column
rows = []
for i in range(len(data)):
    if data['date_added'].iloc[i] == "":
        rows.append(i)
#print(rows)

# examine those rows to confirm null state
data.loc[rows, :]

# extracting months added and years added
year_added = []
month_added = []
for i in range(len(data)):
    # replacing NaN values with 0
    if i in rows:
        month_added.append(0)
        year_added.append(0)
    else:
        date = data['date_added'].iloc[i].split(" ")
        year_added.append(int(date[2]))
        month_added.append(date[0])

#month_added.clear()
#year_added.clear()

In [ ]:
from datetime import datetime as dt

# turning month names into month numbers
for i,mth in enumerate(month_added):
    if mth != 0:
        datetime_obj = dt.strptime(mth, "%B")
        month_number = datetime_obj.month
        month_added[i] = month_number

# checking all months and years
print(set(month_added))
print(set(year_added))

# inserting the month and year columns into the dataset
data.insert(7,"month_added",month_added,allow_duplicates=True)
data.insert(8,"year_added",year_added,allow_duplicates=True)

### Creating 2 datasets from existing one
#### type : TV Show & Movie

In [11]:
# separating original dataset to tv show and movie dataset respectively
shows = []
films = []

# looping through the dataset to identify rows that are TV shows and films
for i in range(len(data)):
    if data['type'].iloc[i] == 'TV Show':
        shows.append(i)
    else:
        films.append(i)

# grouping rows that are TV shows and films
netflix_shows = data.loc[shows,:]
netflix_films = data.loc[films,:]

# reseting the index of the new datasets
netflix_shows = netflix_shows.set_index([pd.Index(range(0, len(netflix_shows)))])
netflix_films = netflix_films.set_index([pd.Index(range(0, len(netflix_films)))])

In [12]:
#another method - forms rangeIndex instead of IntIndex
# netflix_films.index = pd.Index(range(0,len(netflix_films)))
# print(type(netflix_shows.index))
# print(type(netflix_films.index))

In [13]:
# get length of movie or number of seasons of show
def getduration(data):
    count = 0
    duration = []
    for value in data:
        if type(value) is float:
            duration.append(0)
        else:
            values = value.split(" ")
            duration.append(values[0])
    return duration

# inserting new duration type column for shows (renamed column)
netflix_shows.insert(11,'seasons',getduration(netflix_shows['duration']))
netflix_shows.drop(['duration'],axis=1,inplace=True)

# inserting new duration type column for films (renamed column)
netflix_films.insert(11, 'length', getduration(netflix_films['duration']))
netflix_films = netflix_films.drop(['duration'], axis = 1)

### Checking corrupted values(if any)

##### Checking unique values of columns like country, rating, listed_in

title, director, and cast will have large number of unique values so skipping them

In [14]:
netflix_shows['rating'].unique()

array(['TV-MA', 'TV-14', 'TV-Y7', 'TV-PG', 'TV-Y', 'TV-G', 'R', 'NR', '',
       'TV-Y7-FV'], dtype=object)

In [15]:
netflix_films['rating'].unique()  
#contains NR(Not rated) and UR(unRated); which means the same
# contains mins 

array(['PG-13', 'PG', 'TV-MA', 'TV-PG', 'TV-14', 'TV-Y', 'R', 'TV-G',
       'TV-Y7', 'G', 'NC-17', '74 min', '84 min', '66 min', 'NR', '',
       'TV-Y7-FV', 'UR'], dtype=object)

In [16]:
incorrect_ratings = ['74 min', '84 min', '66 min']
weirdRowsIndex = []

for i in range(len(netflix_films)):
    if netflix_films['rating'].iloc[i] in incorrect_ratings:
        weirdRowsIndex.append(i)
        print(netflix_films.iloc[i])
        print("")


show_id                                                     s5542
type                                                        Movie
title                                             Louis C.K. 2017
director                                               Louis C.K.
cast                                                   Louis C.K.
country                                             United States
date_added                                          April 4, 2017
release_year                                                 2017
rating                                                     74 min
listed_in                                                  Movies
length                                                          0
description     Louis C.K. muses on religion, eternal love, gi...
Name: 3562, dtype: object

show_id                                                     s5795
type                                                        Movie
title                                       Louis

#### This weird rating actually belongs to the length column. Hence, replacing the values.

In [17]:
#Correction
for i in weirdRowsIndex:
    values = netflix_films['rating'].iloc[i].split(" ")
    length = values[0]
    netflix_films['length'].iloc[i] = length
    netflix_films['rating'].iloc[i] = "NR"
    
#Checking the entries now
for i in weirdRowsIndex:
    print(netflix_films.iloc[i])

show_id                                                     s5542
type                                                        Movie
title                                             Louis C.K. 2017
director                                               Louis C.K.
cast                                                   Louis C.K.
country                                             United States
date_added                                          April 4, 2017
release_year                                                 2017
rating                                                         NR
listed_in                                                  Movies
length                                                         74
description     Louis C.K. muses on religion, eternal love, gi...
Name: 3562, dtype: object
show_id                                                     s5795
type                                                        Movie
title                                       Louis 

G:\Downloads\anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


#### Changing UR to NR for consistency

In [18]:
for i in range(len(netflix_films)):
    if netflix_films['rating'].iloc[i] == 'UR':
        #netflix_films['rating'].iloc[i] = 'NR'
        netflix_films.loc[i,"rating"] = 'NR'           #indexing better than chaining

#verifying unique values
netflix_films['rating'].unique()


array(['PG-13', 'PG', 'TV-MA', 'TV-PG', 'TV-14', 'TV-Y', 'R', 'TV-G',
       'TV-Y7', 'G', 'NC-17', 'NR', '', 'TV-Y7-FV'], dtype=object)

#### Extract unique country values

In [19]:
# function to get unique values of a column
def getUnique(data):
    unique_values = set()
    for value in data:
        if type(value) is float:
            unique_values.add(None)
            #print("None")
        else:
            split_value = value.split(",")
            for i in split_value:
                #Comma is appended after few counties (eg:'United States,')
                unique_values.add(i.strip())
    return list(unique_values)


In [20]:
# fetching unique countries from main dataset
unique_countries = getUnique(data['country'])
#unique_countries 

#### Comparing Countries with iso package to find inconsistencies 


In [21]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install iso3166

from iso3166 import countries
#countries.get('us')

In [22]:
# fetch countries list from iso3166  
countries_list = []
for c in countries:
    countries_list.append(c.name)

#countries_list

In [23]:
# compare to find differences between the countries we have and the correct ones
rem_countries = []
for i in unique_countries:
    if i not in countries_list:
        rem_countries.append(i)
        
rem_countries

['',
 'Vietnam',
 'Taiwan',
 'Venezuela',
 'United States',
 'Palestine',
 'Iran',
 'West Germany',
 'Vatican City',
 'South Korea',
 'Czech Republic',
 'United Kingdom',
 'Turkey',
 'Soviet Union',
 'Russia',
 'Syria',
 'East Germany']

#### There are few inconsistencies here:

There’s both the Soviet Union and Russia

There’s both the West/East Germany and Germany

Countries starting from ', ' (eg: ', South Korea', etc)


In [24]:
# check if countries go by any other name
for c in countries:
    cname = c.name
    if cname.find('Venezuela')!= -1:
        print(cname)

Venezuela, Bolivarian Republic of


In [25]:
# converting soviet union to russia and east/west germany to germany
for i in range(len(data)):
    if data['country'].iloc[i] != "":
        countries = data['country'].iloc[i].split(", ")
        for j in range(len(countries)):
            if "Germany" in countries[j]:
                countries[j] = "Germany"
            elif "Soviet Union" in countries[j]:
                countries[j] = "Russia"
        data['country'].iloc[i] = ", ".join(countries)
    #print(data['country'].iloc[i])


In [26]:
# correcting instances where country starting from ', '
for i in range(len(data)):
    if data['country'].iloc[i].startswith(','):
        data.loc[i,"country"] = data.loc[i,"country"].strip(', ')         
        print(data.iloc[i])

show_id                                                      s194
type                                                      TV Show
title                                                        D.P.
director                                                         
cast            Jung Hae-in, Koo Kyo-hwan, Kim Sung-kyun, Son ...
country                                               South Korea
date_added                                        August 27, 2021
release_year                                                 2021
rating                                                      TV-MA
duration                                                 1 Season
listed_in                       International TV Shows, TV Dramas
description     A young private’s assignment to capture army d...
Name: 193, dtype: object
show_id                                                      s366
type                                                        Movie
title                                             E

In [27]:
# checking countries
for i in range(len(data)):
    print(data['country'].iloc[i])

IndentationError: expected an indented block (Temp/ipykernel_3916/686672179.py, line 3)

### Checking Genre now

In [28]:
# getting unique film genres
unique_genres_films = getUnique(netflix_films['listed_in'])
unique_genres_films

['Dramas',
 'Cult Movies',
 'International Movies',
 'Thrillers',
 'Action & Adventure',
 'Anime Features',
 'LGBTQ Movies',
 'Romantic Movies',
 'Movies',
 'Classic Movies',
 'Stand-Up Comedy',
 'Independent Movies',
 'Sci-Fi & Fantasy',
 'Children & Family Movies',
 'Documentaries',
 'Sports Movies',
 'Music & Musicals',
 'Horror Movies',
 'Faith & Spirituality',
 'Comedies']

In [29]:
# getting unique show genres
unique_genres_shows = getUnique(netflix_shows['listed_in'])
unique_genres_shows

['Korean TV Shows',
 'TV Shows',
 'TV Dramas',
 'Stand-Up Comedy & Talk Shows',
 'International TV Shows',
 'Reality TV',
 'Romantic TV Shows',
 'TV Horror',
 'Crime TV Shows',
 'Classic & Cult TV',
 'Docuseries',
 'Spanish-Language TV Shows',
 'TV Sci-Fi & Fantasy',
 "Kids' TV",
 'TV Thrillers',
 'Anime Series',
 'Science & Nature TV',
 'TV Mysteries',
 'British TV Shows',
 'Teen TV Shows',
 'TV Action & Adventure',
 'TV Comedies']

In [30]:
# checking for TV shows
# replace netflix_shows with netflix_films to check for moviescount = 0
NoGenre = []
count=0
for i,value in enumerate(netflix_shows['listed_in']):
    genres = value.split(", ")
    if "TV Shows" in genres:
        count += 1
        NoGenre.append(i)
        
print("count %s" %count)
print("index %s" %NoGenre)

count 16
index [59, 110, 272, 286, 452, 599, 991, 1432, 1548, 1808, 1840, 2107, 2160, 2190, 2465, 2559]


In [31]:
## printing the first 5 rows of all rows that have TV Shows as its genre
netflix_shows.iloc[NoGenre[0:5]]

,show_id,type,title,director,cast,country,date_added,release_year,rating,listed_in,seasons,description
59,s149,TV Show,HQ Barbers,Gerhard Mostert,"Hakeem Kae-Kazim, Chioma Omeruah, Orukotan Ade...",,"September 1, 2021",2020,TV-14,TV Shows,1,When a family run barber shop in the heart of ...
110,s298,TV Show,Navarasa,"Bejoy Nambiar, Priyadarshan, Karthik Narain, V...","Suriya, Vijay Sethupathi, Revathy, Prakash Raj...",India,"August 6, 2021",2021,TV-MA,TV Shows,1,"From amusement to awe, the nine human emotions..."
272,s727,TV Show,Metallica: Some Kind of Monster,"Joe Berlinger, Bruce Sinofsky","James Hetfield, Lars Ulrich, Kirk Hammett, Rob...",United States,"June 13, 2021",2014,TV-MA,TV Shows,1,This collection includes the acclaimed rock do...
286,s772,TV Show,Pretty Guardian Sailor Moon Eternal The Movie,Chiaki Kon,"Kotono Mitsuishi, Hisako Kanemoto, Rina Satou,...",,"June 3, 2021",2021,TV-14,TV Shows,1,When a dark power enshrouds the Earth after a ...
452,s1332,TV Show,Five Came Back: The Reference Films,,,United States,"February 9, 2021",1945,TV-MA,TV Shows,1,This collection includes 12 World War II-era p...


##### “TV Shows” and “Movies” genre was used to signify that they didn’t have a genre. We can either choose to exclude or include it in our analysis. 

### Join the cleaned datasets together to create another dataset 

In [32]:
dfs= [netflix_films,netflix_shows]
new_df = pd.concat(dfs)

In [33]:
# Checking null values 
res = new_df.isnull().mean().sort_values(ascending = False)
res

seasons         0.696151
length          0.303849
show_id         0.000000
type            0.000000
title           0.000000
director        0.000000
cast            0.000000
country         0.000000
date_added      0.000000
release_year    0.000000
rating          0.000000
listed_in       0.000000
description     0.000000
dtype: float64

We are getting null values for seasons and length since TV shows doesnot have length values and Films don't have seasons.

In [34]:
# HANDLING NAN
# replace 'seasons' with 'length' to set zero in place of NaN
def NullHandling(data):
    count = 0
    for i in range(len(data)):
        if type(data.iloc[i]) is float:
            count+=1
            data.iloc[i] = '0'
    print(count)
            #new_df['length'].iloc[i] = '0'
        #print(new_df.iloc[i])
NullHandling(new_df['seasons'])        
NullHandling(new_df['length'])        

G:\Downloads\anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


6131
2676


In [35]:
#checking count of NaN values actioned
new_df['seasons'].isnull().sum()

0

In [ ]:
# Exporting cleaned data
# import os  
# os.makedirs('C:/Users/user/NOTEBOOKS/EDA & Vizualization', exist_ok=True)  
# CleanNetflixDataset.to_csv('C:/Users/user/NOTEBOOKS/EDA & Vizualization/Cleaned_netflix_titles.csv') 

No NULL values remain now

### Data has been cleaned. It is ready for analysis.

In [36]:
new_df.describe()

,release_year
count,8807.000000
mean,2014.180198
std,8.819312
min,1925.000000
25%,2013.000000
50%,2017.000000
75%,2019.000000
max,2021.000000


In [37]:
# Stats of whole netflix dataset
new_df.describe(include='object').T

,count,unique,top,freq
show_id,8807,8807,s1,1
type,8807,2,Movie,6131
title,8807,8806,Consequences,2
director,8807,4529,,2634
cast,8807,7693,,825
country,8807,749,United States,2818
date_added,8807,1715,"January 1, 2020",110
rating,8807,14,TV-MA,3207
listed_in,8807,514,"Dramas, International Movies",362
length,8807,206,0,2676


In [38]:
# Stats of netflix shows
netflix_shows.describe(include='object').T

,count,unique,top,freq
show_id,2676,2676,s2,1
type,2676,1,TV Show,2676
title,2676,2676,Blood & Water,1
director,2676,223,,2446
cast,2676,2284,,350
country,2676,197,United States,760
date_added,2676,1013,"July 6, 2021",31
rating,2676,10,TV-MA,1145
listed_in,2676,236,Kids' TV,220
seasons,2676,15,1,1793


In [39]:
new_df['seasons'].unique()

array(['0', '2', '1', '9', '4', '5', '3', '6', '7', '10', '8', '17', '13',
       '15', '12', '11'], dtype=object)

In [40]:
# Which type are mostly aired tv show or movie
new_df["type"].value_counts()

Movie      6131
TV Show    2676
Name: type, dtype: int64

In [41]:
new_df['country'].mode()

0    United States
dtype: object

In [42]:
# Top 10 counties who produced movies and tv shows
a=new_df["country"].value_counts().head(10)
a

United States     2818
India              972
                   831
United Kingdom     419
Japan              245
South Korea        199
Canada             181
Spain              145
France             124
Mexico             110
Name: country, dtype: int64

In [43]:
new_df['year_added'].value_counts()

KeyError: 'year_added'

In [44]:
netflix_films['listed_in'].value_counts()

Dramas, International Movies                        362
Documentaries                                       359
Stand-Up Comedy                                     334
Comedies, Dramas, International Movies              274
Dramas, Independent Movies, International Movies    252
                                                   ... 
Sci-Fi & Fantasy                                      1
Sports Movies                                         1
Children & Family Movies, Comedies, Cult Movies       1
Cult Movies, Dramas, Music & Musicals                 1
Cult Movies, Dramas, Thrillers                        1
Name: listed_in, Length: 278, dtype: int64